<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [4]</a>'.</span>

In [1]:
# Parameters
msgs = "Ran from Airflow at 2021-06-24T21:22:32.224381+00:00!"


In [2]:
import pandas as pd
import mysql.connector
import os
#import json
import datetime

In [3]:
with open("/home/kali/project/input_config.txt") as f:
    input_file=f.readline()
print("will process:",input_file)

will process: /home/kali/project/input/log_meetingAttendanceReport(General).csv


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [4]:
meta_data=[]
log_call=[]
lines=[]
with open(input_file,encoding="utf-16") as f:
    lines=f.readlines()
meta_data=lines[0:5]
log_call=lines[6:]

UnicodeError: UTF-16 stream does not start with BOM

In [ ]:
course = meta_data[2].replace("Titre de la réunion\t","").replace("\n","")
course

In [ ]:
with open(f"/home/kali/project/output/log_{input_file.split('/')[-1]}","w") as f:
    for value in log_call:
        f.write(value)
    f.close()
df_log = pd.read_csv(f"/home/kali/project/output/log_{input_file.split('/')[-1]}",sep="\t")
df_log

In [ ]:
start_str = meta_data[3].replace("Heure de début de la réunion\t","").replace("\n","").replace(" à","")
end_str   = meta_data[4].replace("Heure de fin\t","").replace("\n","").replace(" à","")
start_course = datetime.datetime.strptime(start_str, '%d/%m/%Y %H:%M:%S')
end_course = datetime.datetime.strptime(end_str, '%d/%m/%Y %H:%M:%S')

In [ ]:
df_ds = pd.read_excel("/home/kali/project/lists_excel/Liste des 2èmes années DS.xlsx")
#df_dse = pd.read_excel("/home/kali/notebook/project2/lists_excel/Liste 2A DSE.xlsx")

In [ ]:
mydb = mysql.connector.connect(database="airflow_db", user="db_kali", password="123456789", host="localhost")

In [ ]:
#mycursor = mydb.cursor()
#table="fact_table"
#mycursor.execute(f"SELECT * FROM information_schema.columns WHERE table_schema = 'biabsence' AND table_name   = '{table}' ;")
#myresult = mycursor.fetchall()
#for value in myresult:
#    print(value[3],":",value[7])

In [ ]:
def add_student(name,gender,email,course,major):
    mydb = mysql.connector.connect(database="airflow_db", user="db_kali", password="123456789", host="localhost") 
    mycursor = mydb.cursor()
    sql1 = "SELECT * FROM students where email = %s;"
    vals1 = (email,)
    mycursor.execute(sql1, vals1)
    myresult = mycursor.fetchall()
    if len(myresult) != 0:
        return myresult[0][0]
    if len(myresult) == 0:
        sql = "INSERT INTO students (name,gender,email,course,major) VALUES (%s,%s,%s,%s,%s);"
        val = (name,gender,email,course,major)
        mycursor.execute(sql, val)
        mydb.commit()
        # print(mycursor.rowcount, "record inserted.")
        sql3 = "SELECT * FROM students where email = %s;"
        vals3 = (email,)
        mycursor.execute(sql3, vals3)
        myresult = mycursor.fetchall()
        return myresult[0][0]
    
def add_professor(name,email):
    mydb = mysql.connector.connect(database="airflow_db", user="db_kali", password="123456789", host="localhost") 
    mycursor = mydb.cursor()
    sql1 = "SELECT * FROM professors where email = %s;"
    vals1 = (email, )
    mycursor.execute(sql1, vals1)
    myresult = mycursor.fetchall()
    if len(myresult) != 0:
        return myresult[0][0]
    if len(myresult) == 0:
        sql = "INSERT INTO professors (name,email) VALUES (%s,%s);"
        val = (name,email,)
        mycursor.execute(sql, val)
        mydb.commit()
        # print(mycursor.rowcount, "record inserted.")
        sql3 = "SELECT * FROM professors where email = %s;"
        vals3 = (email, )
        mycursor.execute(sql1, vals1)
        myresult = mycursor.fetchall()
        return myresult[0][0]
    
def add_time(hour,day,month,year,date):
    mydb = mysql.connector.connect(database="airflow_db", user="db_kali", password="123456789", host="localhost")
    mycursor = mydb.cursor()
    sql1 = "SELECT * FROM time WHERE year = %s and month=%s and day=%s and hour=%s;"
    vals1 = (year, month, day, hour, )
    mycursor.execute(sql1, vals1)
    myresult = mycursor.fetchall()
    if len(myresult) != 0:
        return myresult[0][0]
    if len(myresult) == 0:
        sql = "INSERT INTO time (hour,day,month,year,date) VALUES (%s,%s,%s,%s,%s);"
        val = (hour,day,month,year,date)
        mycursor.execute(sql, val)
        mydb.commit()
        # print(mycursor.rowcount, "record inserted.")
        sql3 = "SELECT * FROM time WHERE year = %s and month=%s and day=%s and hour=%s;"
        vals3 = (year, month, day, hour, )
        mycursor.execute(sql3, vals3)
        myresult = mycursor.fetchall()
        return myresult[0][0]

In [ ]:
hour = start_course.hour
day  = start_course.day
month= start_course.month
year = start_course.year
date = start_course.strftime("%Y-%m-%d %H:%M:%S")
id_time=add_time(hour,day,month,year,date)
id_time

In [ ]:
#ADD DS TO STUDENTS TABLE
for index, row in df_ds.iterrows():
    major="DS"
    name,gender,email,course,major = row["Nom"],None,row["email"],course,major
    id_student=add_student(name,gender,email,course,major)
    print("student:",id_student)

In [ ]:
#ADD STUDENTS FROM LOG TO TABLE
for index, row in df_log.iterrows():
    if row["Rôle"]!="Organisateur":
        result = df_ds[df_ds["email"]==row["E-mail"]]
        if(len(result) > 0):
            major="DS"
        else:
            major="DSE"
        name,gender,email,course,major = row["Nom complet"],None,row["E-mail"],course,major
        id_student=add_student(name,gender,email,course,major)
        print("student:",id_student)
    else:
        name,email = row["Nom complet"],row["E-mail"]
        id_prof = add_professor(name,email)
        print("professor:",id_prof)

In [ ]:
groups = df_log.groupby("E-mail")

In [ ]:
mydb = mysql.connector.connect(database="airflow_db", user="db_kali", password="123456789", host="localhost")
mycursor = mydb.cursor()
mycursor.execute("SELECT * FROM students")
myresult = mycursor.fetchall()
emails=[]
for result in myresult:
    emails.append(result[3])

In [ ]:
print("\n".join(emails))

In [ ]:

for email in emails:
    name,gender,email,course,major = None,None,email,course,None
    id_student=add_student(name,gender,email,course,major)
    try:
        data = groups.get_group(email).reset_index()
        nb_abs = 0
        #DURATION
        duration=0
        got_first_start=False
        for index, row in data.iterrows():
            start_str = row["Heure d'arrivée"].replace(" à","")
            end_str   = row["Heure de départ"].replace(" à","")
            start = datetime.datetime.strptime(start_str, '%d/%m/%Y %H:%M:%S')
            end = datetime.datetime.strptime(end_str, '%d/%m/%Y %H:%M:%S')
            duration = duration + ((end-start).total_seconds() / 60.0)
            if not got_first_start:
                got_first_start = True
                first_start = start
        if duration > 120:
            duration=120
        #NB LATES  
        if ((first_start-start_course).total_seconds() / 60.0) > 15:
            nb_lates = 1
        else:
            nb_lates = 0
        
        nb_pres = 1
        
        #ABS DURATION
        abs_duration = 120-duration
        
        #DURATION IN INTEGER IN DB!
        duration = int(duration)
        
        #is student
        print("student:",id_student)
        if duration <= 15:
            nb_abs = 1
            nb_lates=0
            nb_pres=0
    
    except KeyError as e:
        print("not found",e)
        nb_abs = 1
        abs_duration = 120
        nb_lates=0
        nb_pres=0
        duration=0
    #insert to data mart now
    try:
        mydb = mysql.connector.connect(database="airflow_db", user="db_kali", password="123456789", host="localhost")
        mycursor = mydb.cursor()
        sql = "INSERT INTO fact_table (id_time,id_prof,id_student,nb_abs,abs_duration,nb_lates,nb_pres,duration) VALUES (%s,%s,%s,%s,%s,%s,%s,%s);"
        val = (id_time,id_prof,id_student,nb_abs,abs_duration,nb_lates,nb_pres,duration)
        mycursor.execute(sql, val)
        mydb.commit()
    except :
        continue

In [ ]:
os.system(f'mv "{input_file}" "/home/kali/project/processed/"')